# Initialization

In [1]:
import sys
import tensorflow as tf
import pandas as pd
import numpy as np
from joblib import dump, load
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

print (sys.version)
print(tf.__version__)
if tf.test.gpu_device_name(): 
    print(tf.test.is_gpu_available())
    print(tf.test.gpu_device_name())
else:
   print("Please install GPU version of TF")

3.6.13 |Anaconda, Inc.| (default, Mar 16 2021, 11:37:27) [MSC v.1916 64 bit (AMD64)]
1.15.5
True
/device:DML:0


In [2]:
df = pd.read_csv("result.csv")
input_dim = 540 + 32 + 211

In [4]:
df = pd.read_csv("bitsDataset.csv")
input_dim = 32 
df=df.replace(to_replace="Small",value="0")
df=df.replace(to_replace="Big",value="1")

In [5]:
df

,Function,Class,Bit 31,Bit 30,Bit 29,Bit 28,Bit 27,Bit 26,Bit 25,Bit 24,...,Bit 9,Bit 8,Bit 7,Bit 6,Bit 5,Bit 4,Bit 3,Bit 2,Bit 1,Bit 0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4100663,1456381527,1,0,1,0,1,0,1,1,0,...,1,0,0,1,0,1,0,1,1,1
4100664,3194513099,1,1,0,1,1,1,1,1,0,...,1,0,1,1,0,0,1,0,1,1
4100665,1127630112,1,0,1,0,0,0,0,1,1,...,0,1,0,0,1,0,0,0,0,0
4100666,4245509472,1,1,1,1,1,1,1,0,1,...,0,1,0,1,1,0,0,0,0,0


In [ ]:
def wrong_predictions(model,name):
    predictions = np.argmax(model.predict(X_test), axis=-1)
    wrong_predictions = X_test[predictions != y_test].index
    wrong_elements = df.iloc[wrong_predictions].loc[:,['Function','Class']]
    wrong_elements.to_csv('wrong_%s.csv'%name, index=False)

In [4]:
%%time

bitsColsX = [ 'x{}'.format(i) for i in range(540) ]
bitsColsR = [ 'r{}'.format(i) for i in range(32) ]
bitsColsP = [ 'p{}'.format(i) for i in range(211) ]
X = df.loc[:, bitsColsX + bitsColsR + bitsColsP]
Y = df.loc[:, 'Class']
#15% train 85% test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.85, random_state=0)

#We initialize data structures and the regularization parameter.
regul_param = 10.0 ** -2

train_scores = []
test_scores = []
classifiers = []
clf_name = []
clf_topology = []

Wall time: 47.9 s


In [43]:
%%time

bitsColsBit = ['Bit'.format(i) for i in range(32) ]
X = df.loc[:, ['Bit 31','Bit 30','Bit 29','Bit 28','Bit 27','Bit 26','Bit 25','Bit 24','Bit 23','Bit 22','Bit 21','Bit 20','Bit 19','Bit 18','Bit 17','Bit 16','Bit 15','Bit 14','Bit 13','Bit 12', 'Bit 11','Bit 10','Bit 9','Bit 8','Bit 7','Bit 6','Bit 5','Bit 4','Bit 3','Bit 2','Bit 1','Bit 0']]
Y = df.loc[:, 'Class']
#15% train 85% test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.85, random_state=0)

#We initialize data structures and the regularization parameter.
regul_param = 10.0 ** -2

train_scores = []
test_scores = []
classifiers = []
clf_name = []
clf_topology = []

Wall time: 1.9 s


In [26]:
X

,Bit 31,Bit 30,Bit 29,Bit 28,Bit 27,Bit 26,Bit 25,Bit 24,Bit 23,Bit 22,...,Bit 9,Bit 8,Bit 7,Bit 6,Bit 5,Bit 4,Bit 3,Bit 2,Bit 1,Bit 0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4100663,0,1,0,1,0,1,1,0,1,1,...,1,0,0,1,0,1,0,1,1,1
4100664,1,0,1,1,1,1,1,0,0,1,...,1,0,1,1,0,0,1,0,1,1
4100665,0,1,0,0,0,0,1,1,0,0,...,0,1,0,0,1,0,0,0,0,0
4100666,1,1,1,1,1,1,0,1,0,0,...,0,1,0,1,1,0,0,0,0,0


# Classifier Definitions

## 100 Neuron Classifiers

In [ ]:
clf100 = Sequential()
clf100.add(Dense(100, input_dim=input_dim, activation='relu'))
clf100.add(Dense(1, activation='sigmoid'))

clf100.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf100)
clf_name.append("clf100")
clf_topology.append("100")

In [ ]:
clf5050 = Sequential()
clf5050.add(Dense(50, input_dim=input_dim, activation='relu'))
clf5050.add(Dense(50, activation='relu'))
clf5050.add(Dense(1, activation='sigmoid'))

clf5050.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf5050)
clf_name.append("clf5050")
clf_topology.append("50-50")

In [ ]:
clf80 = Sequential()
clf80.add(Dense(80, input_dim=input_dim, activation='relu'))
clf80.add(Dense(15, activation='relu'))
clf80.add(Dense(5, activation='relu'))
clf80.add(Dense(1, activation='sigmoid'))

clf80.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf80)
clf_name.append("clf80")
clf_topology.append("80-15-5")

In [ ]:
clf70 = Sequential()
clf70.add(Dense(70, input_dim=input_dim, activation='relu'))
clf70.add(Dense(25, activation='relu'))
clf70.add(Dense(5, activation='relu'))
clf70.add(Dense(1, activation='sigmoid'))

clf70.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf70)
clf_name.append("clf70")
clf_topology.append("70-25-5")

In [ ]:
clf525 = Sequential()
clf525.add(Dense(50, input_dim=input_dim, activation='relu'))
clf525.add(Dense(25, activation='relu'))
clf525.add(Dense(25, activation='relu'))
clf525.add(Dense(1, activation='sigmoid'))

clf525.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf525)
clf_name.append("clf525")
clf_topology.append("50-25-25")

In [ ]:
clf530 = Sequential()
clf530.add(Dense(50, input_dim=input_dim, activation='relu'))
clf530.add(Dense(30, activation='relu'))
clf530.add(Dense(10, activation='relu'))
clf530.add(Dense(10, activation='relu'))
clf530.add(Dense(1, activation='sigmoid'))

clf530.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf530)
clf_name.append("clf530")
clf_topology.append("50-30-10-10")

## 200 Neuron Classifiers

In [ ]:
clf200 = Sequential()
clf200.add(Dense(200, input_dim=input_dim, activation='relu'))
clf200.add(Dense(1, activation='sigmoid'))

clf200.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf200)
clf_name.append("clf200")
clf_topology.append("200")

In [ ]:
clf155 = Sequential()
clf155.add(Dense(100, input_dim=input_dim, activation='relu'))
clf155.add(Dense(50, activation='relu'))
clf155.add(Dense(50, activation='relu'))
clf155.add(Dense(1, activation='sigmoid'))

clf155.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf155)
clf_name.append("clf155")
clf_topology.append("100-50-50")

In [16]:
clf160 = Sequential()
clf160.add(Dense(160, input_dim=input_dim, activation='relu'))
clf160.add(Dense(25, activation='relu'))
clf160.add(Dense(10, activation='relu'))
clf160.add(Dense(5, activation='relu'))
clf160.add(Dense(1, activation='sigmoid'))

clf160.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf160)
clf_name.append("clf160")
clf_topology.append("160-25-10-5")

In [44]:
clfd160 = Sequential()
clfd160.add(Dense(160, input_dim=input_dim, activation='relu'))
clfd160.add(Dropout(0.1))
clfd160.add(Dense(25, activation='relu'))
clfd160.add(Dropout(0.1))
clfd160.add(Dense(10, activation='relu'))
clfd160.add(Dropout(0.1))
clfd160.add(Dense(5, activation='relu'))
clfd160.add(Dense(1, activation='sigmoid'))

clfd160.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clfd160)
clf_name.append("clfd160")
clf_topology.append("160-25-10-5")

## 800 Neuron Classifiers

In [45]:
clfd1k = Sequential()
clfd1k.add(Dense(600, input_dim=input_dim, activation='relu'))
clfd1k.add(Dropout(0.1))
clfd1k.add(Dense(160, activation='relu'))
clfd1k.add(Dropout(0.1))
clfd1k.add(Dense(25, activation='relu'))
clfd1k.add(Dropout(0.1))
clfd1k.add(Dense(10, activation='relu'))
clfd1k.add(Dropout(0.1))
clfd1k.add(Dense(5, activation='relu'))
clfd1k.add(Dense(1, activation='sigmoid'))

clfd1k.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clfd1k)
clf_name.append("clfd1k")
clf_topology.append("600-160-25-10-5")

## 1600 Neuron Classifiers

In [46]:
clfd1k6 = Sequential()
clfd1k6.add(Dense(800, input_dim=input_dim, activation='relu'))
clfd1k6.add(Dropout(0.1))
clfd1k6.add(Dense(600, activation='relu'))
clfd1k6.add(Dropout(0.1))
clfd1k6.add(Dense(160, activation='relu'))
clfd1k6.add(Dropout(0.1))
clfd1k6.add(Dense(25, activation='relu'))
clfd1k6.add(Dropout(0.1))
clfd1k6.add(Dense(10, activation='relu'))
clfd1k6.add(Dropout(0.1))
clfd1k6.add(Dense(5, activation='relu'))
clfd1k6.add(Dense(1, activation='sigmoid'))

clfd1k6.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clfd1k6)
clf_name.append("clfd1k6")
clf_topology.append("800-600-160-25-10-5")

## 2000 Neuron Classifiers

In [17]:
clf2k = Sequential()
clf2k.add(Dense(1200, input_dim=input_dim, activation='relu'))
clf2k.add(Dense(400, activation='relu'))
clf2k.add(Dense(200, activation='relu'))
clf2k.add(Dense(120, activation='relu'))
clf2k.add(Dense(55, activation='relu'))
clf2k.add(Dense(20, activation='relu'))
clf2k.add(Dense(5, activation='relu'))
clf2k.add(Dense(1, activation='sigmoid'))

clf2k.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf2k)
clf_name.append("clf2k")
clf_topology.append("1200-400-200-120-55-20-5-1")

In [47]:
clfd2k = Sequential()
clfd2k.add(Dense(1200, input_dim=input_dim, activation='relu'))
clfd160.add(Dropout(0.1))
clfd2k.add(Dense(400, activation='relu'))
clfd160.add(Dropout(0.1))
clfd2k.add(Dense(200, activation='relu'))
clfd160.add(Dropout(0.1))
clfd2k.add(Dense(120, activation='relu'))
clfd160.add(Dropout(0.1))
clfd2k.add(Dense(55, activation='relu'))
clfd160.add(Dropout(0.1))
clfd2k.add(Dense(20, activation='relu'))
clfd160.add(Dropout(0.1))
clfd2k.add(Dense(5, activation='relu'))
clfd2k.add(Dense(1, activation='sigmoid'))

clfd2k.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifiers.append(clf2k)
clf_name.append("clfd2k")
clf_topology.append("1200-400-200-120-55-20-5-1")

# Training & Testing

In [48]:
%%time
clfnum = 0
for clf in classifiers:
    train = []
    test = []
    runs = 5
    for i in range(runs):
        clfaux= tf.keras.models.clone_model(clf)
        print("Classifier: ", (clf_name[clfnum]), " Run: ", (i+1))
        print("Compiling...")
        clfaux.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        print("Training...")
        clfaux.fit(X_train, y_train, epochs=1500, batch_size=16384,verbose = 0)
        print("Computing train score...")
        tr_score = clfaux.evaluate(X_train, y_train, verbose = 0)[1]
        print(tr_score)
        train.append(tr_score)
        print("Computing test score...")
        ts_score = clfaux.evaluate(X_test, y_test, verbose = 0)[1]
        print(ts_score)
        test.append(ts_score)
        #filename = 'trained_models/'+clf_name[clfnum] +'_v' + str(i+1) +'.joblib'
        #clfaux.save(filename)
        #wrong_predictions(clfaux,'trained_models/'+clf_name[clfnum] +'_v' + str(i+1))
    clfnum += 1    
    train_scores.append(train)
    test_scores.append(test)

Classifier:  clfd160  Run:  1
Compiling...
Training...
Computing train score...
0.97451794
Computing test score...
0.97247165
Classifier:  clfd160  Run:  2
Compiling...
Training...
Computing train score...
0.97657454
Computing test score...
0.97455794
Classifier:  clfd160  Run:  3
Compiling...
Training...
Computing train score...
0.97324663
Computing test score...
0.97143507
Classifier:  clfd160  Run:  4
Compiling...
Training...
Computing train score...
0.969683
Computing test score...
0.9676383
Classifier:  clfd160  Run:  5
Compiling...
Training...
Computing train score...
0.9670948
Computing test score...
0.9654183
Classifier:  clfd1k  Run:  1
Compiling...
Training...
Computing train score...
0.9999984
Computing test score...
0.9922346
Classifier:  clfd1k  Run:  2
Compiling...
Training...
Computing train score...
0.9999967
Computing test score...
0.99176604
Classifier:  clfd1k  Run:  3
Compiling...
Training...
Computing train score...
0.9999902
Computing test score...
0.99240094
Clas

In [9]:
train_scores = []
test_scores = []

In [49]:
print(clf_name)
print(clf_topology)
print(train_scores)
print(test_scores)

['clfd160', 'clfd1k', 'clfd1k6', 'clfd2k']
['160-25-10-5', '600-160-25-10-5', '800-600-160-25-10-5', '1200-400-200-120-55-20-5-1']
[[0.97451794, 0.97657454, 0.97324663, 0.969683, 0.9670948], [0.9999984, 0.9999967, 0.9999902, 0.9999984, 0.9999886], [1.0, 1.0, 1.0, 0.9999984, 1.0], [1.0, 0.4996716, 1.0, 1.0, 1.0]]
[[0.97247165, 0.97455794, 0.97143507, 0.9676383, 0.9654183], [0.9922346, 0.99176604, 0.99240094, 0.9919841, 0.99222624], [0.99117047, 0.991394, 0.9915701, 0.99072576, 0.99079776], [0.9813712, 0.50005794, 0.9816822, 0.9825389, 0.98265016]]


# Results Visualization

In [50]:
train = []
test = []
train_err = []
test_err = []

for i in range(len(clf_name)):
    train.append(np.mean(train_scores[i]))
    train_err.append(np.std(train_scores[i]))
    test.append(np.mean(test_scores[i]))
    test_err.append(np.std(test_scores[i]))
means = pd.DataFrame({'Train': train,'Test': test}, index=clf_topology)
errors = pd.DataFrame({'Train': train_err,'Test': test_err}, index=clf_topology)


In [51]:
means

,Train,Test
160-25-10-5,0.972223,0.970304
600-160-25-10-5,0.999994,0.992122
800-600-160-25-10-5,1.000000,0.991132
1200-400-200-120-55-20-5-1,0.899934,0.885660


In [52]:
errors

,Train,Test
160-25-10-5,3.405900e-03,0.003318
600-160-25-10-5,4.197877e-06,0.000222
800-600-160-25-10-5,6.447228e-07,0.000328
1200-400-200-120-55-20-5-1,2.001314e-01,0.192802


In [ ]:
means = pd.DataFrame({'Train': train,'Test': test}, index=clf_topology)
errors = pd.DataFrame({'Train': train_err,'Test': test_err}, index=clf_topology)

sup = 1.005*max(max(train),max(test))
inf = 0.995*min(min(train),min(test))
ax = means.plx = means.plot.bar(rot=0,yerr=errors,figsize=(12,10),ylim=[inf,sup],capsize=4, xlabel= "Topology", ylabel= "Accuracy")